In [5]:
# import required libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
# Save all parquet files
# !tar chvfz customer_big.tar.gz bigdata2021data/customer_big.parquet/*

In [6]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [7]:
def df_explore(df):
    """prints summary information of df""" 
    print('No. of rows: ', len(df))
    print('*'*30)
    
    for col in df: # print summary of each variables
        print(col)
        print(df[col].unique()[:5])
        print('Column index: ', df.columns.get_loc(col))
        print('No. of unique entries: ',len(df[col].unique()))
        print('Datatype: ', df[col].dtypes)
        print('*'*30)
    
def col_type(df):
    """returns dictionary of column names that have a given datatype"""
    g = df.columns.to_series().groupby(df.dtypes)
    return {k.name: v for k, v in g.groups.items()}

def convert2lower(df):
    """convert all string to lowercase and strips leading/trailing white space""" 
    for col in col_type(df)['object']:
        df[col] = df[col].astype(str).str.lower()
        df[col] = df[col].str.strip()
    return df

def zero_fill_nan(df, cols):
    """Replace NaN with zero"""
    df[cols]=df[cols].fillna(0)
    return df

Note: if you are working on your own computer instead of the JupyterHub, you might have to install pyarrow to be able to open the following parquet files.

In [8]:
# load the customer training data set:
df1 = pd.read_parquet("cust_train.parquet")
# Preprocess df1
df1 = convert2lower(df1)
# df1.info()
# df1.nunique()
df_explore(df1)

No. of rows:  4469
******************************
customer_id_mskd
['df39588796bc20fb01d282213200c944ad38caa06a91a77c821d6396fc856ea5'
 'd56f5632aa40d2661dcc5419def6ede2ed4cca24bd76cdb4ece0d6981e03ccb4'
 'a86efc98b6e70bbb37c12024304f3d7c9eac9e4ecd9559fa02a6e29da5954344'
 '24f154d9bb05a18ba723830b6c41d6eb71a67f14ca73251876eb96de21d41260'
 'bdb58c996ed0b5c9f81461caa8efe914f712e22ffaa6ac38157e8940a821bcb3']
Column index:  0
No. of unique entries:  4469
Datatype:  object
******************************
jurisdiction_code
['ca03' 'ca08' 'ca16']
Column index:  1
No. of unique entries:  3
Datatype:  object
******************************
client_type_aml
['individual']
Column index:  2
No. of unique entries:  1
Datatype:  object
******************************
industry_code_aml
['none']
Column index:  3
No. of unique entries:  1
Datatype:  object
******************************
occupation_code_aml
['108' '342' 'e902' '94' '256']
Column index:  4
No. of unique entries:  235
Datatype:  object
*******

In [39]:
df1.occupation_code_aml.unique()
df1.occupation_code_aml.value_counts()

256     1054
e902     756
e904     616
e901     418
e900     274
        ... 
15         1
75         1
230        1
192        1
244        1
Name: occupation_code_aml, Length: 235, dtype: int64

In [10]:
# load the transaction training data set:
df2 = pd.read_parquet("transaction_train.parquet")
df2 = zero_fill_nan(df2, ['in_amt', 'in_cnt', 'out_amt', 'out_cnt'])

# Analyze data in df2
df_explore(df2)
# '510ad85cdbe68ad98edaf81cbc2fda6fc218d1c4acc53b974df2e85e2cc0d559' in df1['customer_id_mskd'].unique()

No. of rows:  69534
******************************
customer_id_mskd
['4eb76d305d32c1d00cc0d8850abe45ffc49f51f08324ee2c3070e0d59232776b'
 '510ad85cdbe68ad98edaf81cbc2fda6fc218d1c4acc53b974df2e85e2cc0d559'
 '8115b4ba5e267184283a64117e9664deb55159c1bd4e2ddfa5ae89a5aa531168'
 '6c854026065a868b6134d04d906dcfa9af7278ff834b0cbfcee19d9fc248d899'
 'd8cc95339794bf3f84f941b1eba6266d58af1dc7e651782a44c83a03129cb16a']
Column index:  0
No. of unique entries:  2827
Datatype:  object
******************************
month
['2020-02' '2019-04' '2019-05' '2019-06' '2020-01']
Column index:  1
No. of unique entries:  13
Datatype:  object
******************************
in_amt
[ 150.  400.    0. 1200.   20.]
Column index:  2
No. of unique entries:  22485
Datatype:  float64
******************************
in_cnt
[1. 0. 2. 4. 3.]
Column index:  3
No. of unique entries:  58
Datatype:  float64
******************************
out_amt
[1600. 1000.  500.  315.    0.]
Column index:  4
No. of unique entries:  43436
Data

In [15]:
df2.describe()

,in_amt,in_cnt,out_amt,out_cnt
count,6.953400e+04,69534.000000,6.953400e+04,69534.00000
mean,3.273571e+03,1.489516,3.492482e+03,13.64984
std,6.326930e+04,3.121156,4.316777e+04,24.28942
min,0.000000e+00,0.000000,0.000000e+00,0.00000
25%,0.000000e+00,0.000000,1.150000e+02,1.00000
50%,5.000000e+01,1.000000,5.800000e+02,4.00000
75%,1.075380e+03,2.000000,1.888535e+03,15.00000
max,8.333333e+06,172.000000,5.134000e+06,748.00000


In [31]:
df2.month.unique()
#out_count = df2.groupby('out_amt').count()
#out_count.sort_values(by=['out_amt'], ascending = True)

array(['2020-02', '2019-04', '2019-05', '2019-06', '2020-01', '2019-11',
       '2019-08', '2019-09', '2019-07', '2019-12', '2020-03', '2019-10',
       '2020-04'], dtype=object)

In [13]:
# load the large customer data set:
df3 = pd.read_parquet("bigdata2021data/customer_big.parquet")

# this might fail because of memory constraints!

df3.groupby('client_type_aml').head()

FileNotFoundError: [Errno 2] No such file or directory: 'customer_big.parquet'

In [ ]:
# however, you can still load individual parts of that large file as it is partitioned into 200 individual files
df3_0 = pd.read_parquet("bigdata2021data/customer_big.parquet/part-00000-9799628b-5c6c-499c-a655-f339d13ed4c0-c000.snappy.parquet")
df3_0 = convert2lower(df3_0)
# Analyze data in df3_0
df_explore(df3_0)


In [ ]:
df3_0['relationship_type'].unique()

In [ ]:
# load the large transaction data set:
df4 = pd.read_parquet("bigdata2021data/transaction_big.parquet")
df4
# this might fail because of memory constraints!

In [ ]:
# however, you can still load individual parts of that large file as it is partitioned into 122 individual files
df4_0 = pd.read_parquet("bigdata2021data/transaction_big.parquet/part-00000-d7f04269-6d76-4ab2-bd40-0229b3885a23-c000.snappy.parquet")
df4_0 = convert2lower(df4_0)

df_explore(df4_0)


In [ ]:
df4_1 = pd.read_parquet("bigdata2021data/transaction_big.parquet/part-00001-d7f04269-6d76-4ab2-bd40-0229b3885a23-c000.snappy.parquet")
df4_1

## A tiny toy example for saving files:

In [ ]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

In [ ]:
df.to_parquet("filename.parquet")

<div class="alert alert-block alert-info">
<b>Easy Case:</b> Automate an Expert using labelled data
    <br>
    - This makes use of labels provided by experts in the “customer_train” table
</div>

In [181]:
# Supervised ML
# @input: 
# @output: rating (multiclass logistic)

# Algorithm: SGD Classifier 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import make_pipeline

def OneHotEncode_col(df, col_index):
#     col_index = df.columns.get_loc(col_name)
#     @col_index : array
    ind_pass = np.delete(np.arange(0, len(df.columns)), col_index)
    
    c_transform = ColumnTransformer([('onehot', OneHotEncoder(), col_index), ('nothing', 'passthrough', ind_pass)])
    x = df.values
    return c_transform.fit_transform(x).astype(float).toarray()

In [404]:
col_remove_easy = [0, 2, 3, 8, 12, 16, *range(20,26+1)]
df_easy = df1.drop(df1.columns[col_remove_easy], axis=1)
df_easy.fillna(0, inplace=True) # fill nan with 0
X_easy = OneHotEncode_col(df_easy.iloc[:, :-1], [*range(0, 6+1), 14])
y_easy = df_easy['rating'].values # rating column

# shuffle
idx = np.arange(X_easy.shape[0])
np.random.seed(9)
np.random.shuffle(idx)
X_easy = X_easy[idx]
y_easy = y_easy[idx]


# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X_easy, y_easy, test_size=0.33)

clf = make_pipeline(StandardScaler(), SGDClassifier(alpha=0.001, max_iter=100)).fit(X_train, y_train)


In [405]:
clf.decision_function(X_test)
clf.predict(X_test)
y_test

array([3., 2., 3., ..., 2., 1., 1.])

In [421]:
compare = pd.DataFrame({'rating_obs':y_test, 'rating_pred':clf.predict(X_test)}, columns=['rating_obs','rating_pred'])
rating_risk = 3

compare['Correct?'] = np.where(
    compare['rating_obs'] == compare['rating_pred'], 1, 0)
compare['True Positive (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] == rating_risk) & (compare['rating_pred'] == rating_risk), 1, 0)
compare['False Positive (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] != rating_risk) & (compare['rating_pred'] == rating_risk), 1, 0)
compare['True Negative (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] != rating_risk) & (compare['rating_pred'] != rating_risk) & (compare['Correct?'] == 1), 1, 0)
compare['True-ish Negative (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] != rating_risk) & (compare['rating_pred'] != rating_risk) & (compare['Correct?'] == 0), 1, 0)
compare['False Negative (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] == rating_risk) & (compare['rating_pred'] != rating_risk), 1, 0)

for i in range(1, 7):
    col = compare.columns[i+1]
    percent = '{:.1%}'.format(compare[col].sum()/len(compare))
    print(f'{col} : {compare[col].sum()} out of {len(compare)} : {percent}')

Correct? : 1367 out of 1475 : 92.7%
True Positive (3) : 405 out of 1475 : 27.5%
False Positive (3) : 8 out of 1475 : 0.5%
True Negative (3) : 962 out of 1475 : 65.2%
True-ish Negative (3) : 58 out of 1475 : 3.9%
False Negative (3) : 42 out of 1475 : 2.8%


749

<div class="alert alert-block alert-info">
<b>Medium Case:</b> Improve the accuracy by augmenting the data
    <br>
    - A smaller “transaction_train” table is an easy place to start augmenting. This does not correspond one-to-one with the customer table, so you will have to figure out some way to combine them.
</div>

In [199]:
# Supervised ML
# @input: 
# @output: rating (multiclass logistic)

# sort by month to keep temporal series

In [414]:
col_remove_med = [2, 3, 8, 12, 16, *range(20,26+1)]

df_med = df1.drop(df1.columns[col_remove_med], axis=1)
df_med.fillna(0, inplace=True) # fill nan with 0
df_med = pd.merge(df_med, df2, on='customer_id_mskd', how='outer')
df_med = df_med[df_med['month'].notna()] # Remove nan from month
df_med['month'] = pd.to_datetime(df_med.month)
df_med.sort_values(by='month', inplace=True)

dict_date = dict(enumerate(df_med['month'].unique()))
dict_date = dict((v,k) for k,v in dict_date.items())

df_med.replace({"month": dict_date}, inplace=True)

X_med = OneHotEncode_col(df_med, [*range(0, 7+1), 15, 22])
y_med = df_med['rating'].values # rating column

# shuffle
idx = np.arange(X_easy.shape[0])
np.random.seed(7)
np.random.shuffle(idx)
X_med = X_med[idx]
y_med = y_med[idx]

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X_med, y_med, test_size=0.33)

clf = make_pipeline(StandardScaler(), SGDClassifier(alpha=0.001, max_iter=100)).fit(X_train, y_train)


In [415]:
compare = pd.DataFrame({'rating_obs':y_test, 'rating_pred':clf.predict(X_test)}, columns=['rating_obs','rating_pred'])
rating_risk = 3

compare['Correct?'] = np.where(
    compare['rating_obs'] == compare['rating_pred'], 1, 0)
compare['True Positive (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] == rating_risk) & (compare['rating_pred'] == rating_risk), 1, 0)
compare['False Positive (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] != rating_risk) & (compare['rating_pred'] == rating_risk), 1, 0)
compare['True Negative (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] != rating_risk) & (compare['rating_pred'] != rating_risk) & (compare['Correct?'] == 1), 1, 0)
compare['True-ish Negative (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] != rating_risk) & (compare['rating_pred'] != rating_risk) & (compare['Correct?'] == 0), 1, 0)
compare['False Negative (' + str(rating_risk)+ ')'] = np.where(
    (compare['rating_obs'] == rating_risk) & (compare['rating_pred'] != rating_risk), 1, 0)

for i in range(1, 7):
    col = compare.columns[i+1]
    percent = '{:.1%}'.format(compare[col].sum()/len(compare))
    print(f'{col} : {compare[col].sum()} out of {len(compare)} : {percent}')

Correct? : 92.7%
True Positive (3) : 27.5%
False Positive (3) : 0.5%
True Negative (3) : 65.2%
True-ish Negative (3) : 3.9%
False Negative (3) : 2.8%


<div class="alert alert-block alert-info">
<b>Hard Case:</b> Identify accounts without relying on experts
    <br>
    - This is unsupervised learning, and can most likely benefit from as much data as you can manage. There is an awful lot of unlabelled data compared to the labelled stuff.
</div>

In [ ]:
# Undersupervised -> Supervised ML
# @input: 
# @output: clusters (filtered)

# Run supervised ML on each cluster